In [2]:
import pandas as pd
df = pd.read_csv('wfC318/Day_0/Raw_files_csv/combined_behavior_and_s.csv', header=None)

#adding labels to the data
labs = ['frame', 'time', 'odor_valve', 'total_licks', 'total_rewards', 'trial', 
        'valve_open', 'virt_dist', 'total_distance', 'lap', 'trial_type', 'speed']

#name the rest of the columns with neuron index
for i in range(0,255):
    labs.append(str(i))
    
df.columns = labs
print(df.head())

   frame  time  odor_valve  total_licks  total_rewards  trial  valve_open  \
0      1    34           0            0              0      0           0   
1      2    84           0            0              0      0           0   
2      3   133           0            0              0      0           0   
3      4   183           0            0              0      0           0   
4      5   233           0            0              0      0           0   

   virt_dist  total_distance  lap  ...  245     246  247  248  249  250  251  \
0         13               0    0  ...  0.0  0.0000  0.0  0.0  0.0  0.0  0.0   
1         26               0    0  ...  0.0  2.7654  0.0  0.0  0.0  0.0  0.0   
2         36               0    0  ...  0.0  0.0000  0.0  0.0  0.0  0.0  0.0   
3         50               0    0  ...  0.0  0.0000  0.0  0.0  0.0  0.0  0.0   
4         65               0    0  ...  0.0  0.0000  0.0  0.0  0.0  0.0  0.0   

   252  253  254  
0  0.0  0.0  0.0  
1  0.0  0.0  0.0  

In [21]:
#remove all trials after 30
df = df[df["trial"] < 31]

#restart time so it increments within a given trial only
#do not run this cell more than once or it will fuck everything up
trial_prev, time_prev, end_time = 0,0,0
for i, j in df.iterrows(): 
    trial = int(j['trial'])
    time = int(j['time'])
    if trial != trial_prev:
        end_time = time_prev
        print("\n")
    df.at[i,'time'] = time - end_time
    trial_prev = trial
    time_prev = time

In [23]:
#remove all cells where distance exceeds 4,000cm
df = df[df["virt_dist"] < 4000]

In [1]:
import pandas as pd
import numpy as np
import time as t

##########
# Things to set:
#filename = 'wfC318/Day_4/raw_files_csv/combined_behavior_and_s_COPY.csv'
#filename = 'wfC318/Day_4/raw_files_csv/combined_behavior_and_s.csv'
filename = 'wfC318/Day_0/Raw_files_csv/combined_behavior_and_s.csv'


# How many trials we are looking at (ex: 31 for day 0, 34 for day 4)
trial_count = 31
#trial_count = max(lap)  # all of them

##########

data = pd.read_csv(filename, header=None)
#data2 = pd.read_csv(filename, header=None) # to be used for binning, after altering times

def changeTime():
    trial_prev, time_prev, end_time = 0,0,0
    for i, j in data.iterrows():
        trial = int(j[9])
        time = int(j[1])
        if trial != trial_prev:
            end_time = time_prev
        data.at[i,1] = time - end_time
        trial_prev = trial
        time_prev = time
    #print("Times changed")
changeTime()


#print("Preview:", data.head())
length = len(data)
data = data[data[7] <= 4000]  # removing all rows where mouse is licking reward past 4000cm
#print("Removed", length-len(data))


time = data[1]
odor = data[10]
distance = data[7]
lap = data[9]
# neuron data starts at column "13", indexed from 0 so the first column of neuron data is data[12]
cell = data.loc[:, 12:(len(data.columns)-1)]  # neuron data

##### Cleaning up data
# Changing neuron firing data to 0,1 binary
num_of_cols = len(data.columns)

for j in range(12, num_of_cols):  # for all neuron columns
    # setting value to 1 if val>0, 0 otherwise
    data[j].apply(lambda x: 1 if x > 0 else 0)
#####

lap_dictionary = {}
for i in range(0, trial_count):  # for every lap
    lap_dictionary[i] = data.loc[data[9] == i]  # associate all lap data with that lap
#print(lap_dictionary)

def findShortestTrial():
    min_time = 999999999
    max_time = 0
    shortest_trial = lap_dictionary[0]
    longest_trial = lap_dictionary[0]
    time_list = []
    for trial in lap_dictionary:
        #pair = lap_dictionary[trial].iloc[[0, -1]]
        first = lap_dictionary[trial].iloc[0]
        last = lap_dictionary[trial].iloc[-1]

        time = last[1] - first[1]
        time_list.append(time)

        if(time < min_time):
            min_time=time
            shortest_trial = trial
        if (time > max_time):
            max_time = time
            longest_trial = trial

    #print("AVG trial duration:", sum(time_list)/len(time_list))
    return shortest_trial, min_time, longest_trial, max_time

shortest_trial, min_time,longest_trial, max_time = findShortestTrial()
#print("Shortest Trial:", shortest_trial, "Time: ", min_time)
#print("Longest Trial:", longest_trial, "Time: ", max_time)

############
# Determining number of bins
############
num_of_bins = int(min_time // 200)
print("number of bins:", num_of_bins)


# create empty row to store neuron data frequencies
empty_row = {}
for k in range(0, num_of_cols-13):  # 255 or 459 columns for example
    empty_row[k] = 0.0

def findMeanValues():
    flagCounter = 0
    test_dictionary_for_laps = {}
    start = t.time()  # for reference
    for i in lap_dictionary:  # for every trial
        # print("Lap number:", i)
        # average columns in that trial to test (eventually, in bin instead)
        current_row = pd.DataFrame(empty_row, index=[0])  # create pandas dataframe from a row of 0's (reset every lap)
        #print(i)  # count 89~99 trial laps
        for j in range(12, num_of_cols):  # for all neuron columns
            #current_row[j - 12] = data[j].mean()
            current_row[j-12] = lap_dictionary[i][j].mean()  # fill each column spot in that row with the mean of all values in that column
            # note: because we set values to (0,1), this mean becomes frequency relative to that trial

        """
        # print a few for testing
        if flagCounter < 5:
            print(current_row)
            flagCounter += 1
        """
        test_dictionary_for_laps[i] = current_row  # set dictionary item equal to that row

    end = t.time()
    #print("Time:", end-start)
    return test_dictionary_for_laps

test_dictionary_for_laps = findMeanValues()


def changeTime():
    trial_prev, time_prev, end_time = 0,0,0
    for i, j in data.iterrows():
        trial = int(j[9])
        time = int(j[1])
        if trial != trial_prev:
            end_time = time_prev
        data.at[i,1] = time - end_time
        trial_prev = trial
        time_prev = time
    #print("Times changed")
#changeTime()

#print("CHECK:", data.tail())

NEW_lap_dictionary = {}
for i in range(0, trial_count):  # for every lap
    NEW_lap_dictionary[i] = data.loc[data[9] == i]  # associate all lap data with that lap
    #print(NEW_lap_dictionary[i].iloc[:,:13], "\n\n")

def changeTimes():
    flag = 0
    for trial in NEW_lap_dictionary:
        if flag == 1:  # change times after trial 1
            end_of_last_trial = NEW_lap_dictionary[trial-1].iloc[-1,1]
            #print(NEW_lap_dictionary[trial])
            for row in NEW_lap_dictionary[trial]:
                #print("HERE", row)
                curr_time = NEW_lap_dictionary[trial].iat[row,1]
                #print(curr_time)
                data.iat[row,1] = curr_time - end_of_last_trial
                #NEW_lap_dictionary[trial].iat[row,1] = curr_time-end_of_last_trial
        flag = 1
    #print("Times changed")
#changeTimes()


def binning():
    # each key value pair will be (Trial_number, [ [Bin1], [Bin2], [Bin3], ..., [Bin43] ])
    dict_of_all_bins = {}
    for i in NEW_lap_dictionary:  # for every trial
        #print("TEST", NEW_lap_dictionary[i], "\n\n")
        list_of_bins = {}  # each (key,value) pair will be start-time of bin and list of points in bin for each trial
        # indexed from 1
        for bin_number in range(1, num_of_bins+1):
            bin_starting_time = bin_number*200
            last_time = (bin_number-1)*200
            # add all data points from the current trial in lap_dictionary to bin if time is in bin range
            data_points = NEW_lap_dictionary[i].loc[(NEW_lap_dictionary[i][1] < bin_starting_time) & (NEW_lap_dictionary[i][1] >= last_time) ]
            list_of_bins[bin_starting_time] = data_points
            #print(data_points)
        dict_of_all_bins[i] = list_of_bins

    return dict_of_all_bins
binned_dic = binning()
print("binned dic")

#for k,v in ret.items():
#    print("key", k)
#    print("value:   ", v)

#def findRelevantNeurons():
#    for rows in test_dictionary_for_laps:  # there should be 89 or 99, 1 per trial
#        for columns in rows:

#index_of_maxs = []
#for row in test_dictionary_for_laps:
#    index_of_maxs.append(test_dictionary_for_laps[row].idxmax())  # something's messed up here

#print(len(index_of_maxs), index_of_maxs)



number of bins: 43
binned dic


In [3]:
print("binned dic")

i=0
for trial,bin in binned_dic.items():
    if trial != 0:
        break
    times,cells,spikes = [],[],[]
    for time,activities in bin.items():
        #print("time:", time)
        for cell,v in activities.items():
            if cell == 0:
                continue
            #print("cell:", cell)
            activity_sum = 0
            for x in v:
                activity_sum += x
            #print("total_activity", activity_sum)
            times.append(time)
            cells.append(cell)
            spikes.append(activity_sum)

binned dic


In [4]:
data_dic = {}
data_dic["times"] = times
data_dic["cells"] = cells
data_dic["spikes"] = spikes

In [5]:
df = pd.DataFrame.from_dict(data_dic)
print(df.head())

   times  cells  spikes
0    200      1   434.0
1    200      2     0.0
2    200      3     0.0
3    200      4     0.0
4    200      5     0.0


In [6]:
df = df.pivot(index='cells', columns='times', values='spikes')
print(df.head())

times   200     400     600     800     1000    1200    1400    1600    1800  \
cells                                                                          
1      434.0  1233.0  2034.0  2832.0  3634.0  4433.0  5235.0  6035.0  6835.0   
2        0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3        0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     4.0   
4        0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
5        0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

times    2000  ...     6800     7000     7200     7400     7600     7800  \
cells          ...                                                         
1      7634.0  ...  26837.0  27637.0  28440.0  29239.0  30040.0  30842.0   
2         0.0  ...      0.0      0.0      0.0      0.0      0.0      0.0   
3         4.0  ...     24.0     26.0     28.0     28.0     28.0     28.0   
4         0.0  ...      0.0      0.0      0.0      0.0     

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os 
#os.mkdir("./plots")
print("binned dic")

for trial,bin in binned_dic.items():
    #print("trial:", trial)
    times,cells,spikes = [],[],[]
    for time,activities in bin.items():
        #print("time:", time)
        for cell,v in activities.items():
            if cell == 0 or cell > 15:
                continue
            activity_sum = 0
            count = 0
            for x in v:
                activity_sum += x
                count += 1
            mean_activity = activity_sum / count
            #print("time:", time, "cell:", cell, "mean_act:", mean_activity)
            times.append(time)
            cells.append(cell)
            spikes.append(mean_activity)
    data_dic = {}
    data_dic["times"] = times
    data_dic["cells"] = cells
    data_dic["spikes"] = spikes

    df = pd.DataFrame.from_dict(data_dic)
    df = df.pivot(index='cells', columns='times', values='spikes')
    fig, ax = plt.subplots(figsize=(20,20)) 
    plt.title("trial "+str(trial))
    sns.heatmap(df, annot=False)
    plt.savefig("plots/trial"+str(trial)+".png")

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import os 

times,cells,spikes,t = [],[],[],[]
data_dic = {}
for trial,bin in binned_dic.items():
    for time,activities in bin.items():
        #print("time:", time)
        for cell,v in activities.items():
            if cell == 0:
                continue
            activity_sum = 0
            count = 0
            for x in v:
                activity_sum += x
                count += 1
            mean_activity = activity_sum / count
            t.append(trial)
            times.append(time)
            cells.append(cell)
            spikes.append(mean_activity)
    #data_dic["trial"] = trials
    data_dic["trial"] = t
    data_dic["times"] = times
    data_dic["cells"] = cells
    data_dic["spikes"] = spikes

df = pd.DataFrame.from_dict(data_dic)

print(df.head())

   trial  times  cells  spikes
0      0    200      1   108.5
1      0    200      2     0.0
2      0    200      3     0.0
3      0    200      4     0.0
4      0    200      5     0.0


In [18]:
avrg_spike = {}
#there are 31 trials
for i, j in df.iterrows():
    time = int(j["times"])
    cell = int(j["cells"])
    spike = int(j["spikes"])
    if (time, cell) in avrg_spike:
        avrg_spike[(time,cell)] += spike
    else:
        avrg_spike[(time,cell)] = spike

#avrg spike contains the avrg spiking for each (bin, cell) pair
print(avrg_spike)
for k,v in avrg_spike.items():
    avrg_spike[k] = v/31

#df1 = df.copy()
#for i in range(0, 1000):
#    df1["spikes"] = np.random.permutation(df["spikes"].values)

{(200, 1): 3266, (200, 2): 0, (200, 3): 21238, (200, 4): 6828, (200, 5): 465, (200, 6): 0, (200, 7): 56, (200, 8): 1911, (200, 9): 465, (200, 10): 31, (200, 11): 67, (200, 12): 0, (200, 13): 0, (200, 14): 0, (200, 15): 0, (200, 16): 0, (200, 17): 0, (200, 18): 0, (200, 19): 0, (200, 20): 0, (200, 21): 0, (200, 22): 0, (200, 23): 0, (200, 24): 0, (200, 25): 0, (200, 26): 0, (200, 27): 0, (200, 28): 0, (200, 29): 0, (200, 30): 0, (200, 31): 2, (200, 32): 0, (200, 33): 0, (200, 34): 1, (200, 35): 0, (200, 36): 0, (200, 37): 0, (200, 38): 0, (200, 39): 1, (200, 40): 0, (200, 41): 1, (200, 42): 0, (200, 43): 5, (200, 44): 0, (200, 45): 0, (200, 46): 0, (200, 47): 0, (200, 48): 0, (200, 49): 0, (200, 50): 0, (200, 51): 0, (200, 52): 0, (200, 53): 0, (200, 54): 0, (200, 55): 0, (200, 56): 0, (200, 57): 0, (200, 58): 0, (200, 59): 0, (200, 60): 0, (200, 61): 0, (200, 62): 0, (200, 63): 0, (200, 64): 0, (200, 65): 0, (200, 66): 2, (200, 67): 0, (200, 68): 0, (200, 69): 0, (200, 70): 4, (200, 71

In [20]:
for i, j in df.iterrows():
    time = int(j["times"])
    cell = int(j["cells"])
    spike = int(j["spikes"])
    trial = int(j["trial"])
    average = avrg_spike[(time,cell)]
    if spike > (3*average):
        print("trial:", trial, "cell:", cell, "bin:", time)

trial: 0 cell: 7 bin: 200
trial: 0 cell: 11 bin: 200
trial: 0 cell: 34 bin: 200
trial: 0 cell: 168 bin: 200
trial: 0 cell: 170 bin: 200
trial: 0 cell: 220 bin: 200
trial: 0 cell: 7 bin: 400
trial: 0 cell: 11 bin: 400
trial: 0 cell: 54 bin: 400
trial: 0 cell: 7 bin: 600
trial: 0 cell: 11 bin: 600
trial: 0 cell: 54 bin: 600
trial: 0 cell: 156 bin: 600
trial: 0 cell: 184 bin: 600
trial: 0 cell: 206 bin: 600
trial: 0 cell: 7 bin: 800
trial: 0 cell: 11 bin: 800
trial: 0 cell: 24 bin: 800
trial: 0 cell: 58 bin: 800
trial: 0 cell: 68 bin: 800
trial: 0 cell: 70 bin: 800
trial: 0 cell: 98 bin: 800
trial: 0 cell: 100 bin: 800
trial: 0 cell: 156 bin: 800
trial: 0 cell: 170 bin: 800
trial: 0 cell: 263 bin: 800
trial: 0 cell: 7 bin: 1000
trial: 0 cell: 24 bin: 1000
trial: 0 cell: 51 bin: 1000
trial: 0 cell: 52 bin: 1000
trial: 0 cell: 68 bin: 1000
trial: 0 cell: 213 bin: 1000
trial: 0 cell: 214 bin: 1000
trial: 0 cell: 227 bin: 1000
trial: 0 cell: 236 bin: 1000
trial: 0 cell: 7 bin: 1200
trial: 0 c

trial: 1 cell: 214 bin: 8000
trial: 1 cell: 236 bin: 8200
trial: 1 cell: 74 bin: 8400
trial: 1 cell: 77 bin: 8600
trial: 2 cell: 70 bin: 200
trial: 2 cell: 76 bin: 200
trial: 2 cell: 82 bin: 200
trial: 2 cell: 95 bin: 200
trial: 2 cell: 142 bin: 200
trial: 2 cell: 145 bin: 200
trial: 2 cell: 214 bin: 200
trial: 2 cell: 253 bin: 200
trial: 2 cell: 66 bin: 400
trial: 2 cell: 95 bin: 400
trial: 2 cell: 132 bin: 400
trial: 2 cell: 152 bin: 400
trial: 2 cell: 176 bin: 400
trial: 2 cell: 234 bin: 400
trial: 2 cell: 41 bin: 800
trial: 2 cell: 189 bin: 800
trial: 2 cell: 41 bin: 1000
trial: 2 cell: 89 bin: 1000
trial: 2 cell: 134 bin: 1000
trial: 2 cell: 189 bin: 1000
trial: 2 cell: 93 bin: 1200
trial: 2 cell: 134 bin: 1200
trial: 2 cell: 161 bin: 1200
trial: 2 cell: 193 bin: 1200
trial: 2 cell: 219 bin: 1200
trial: 2 cell: 54 bin: 1400
trial: 2 cell: 73 bin: 1400
trial: 2 cell: 74 bin: 1400
trial: 2 cell: 98 bin: 1400
trial: 2 cell: 144 bin: 1400
trial: 2 cell: 150 bin: 1400
trial: 2 cell: 22

trial: 3 cell: 134 bin: 4600
trial: 3 cell: 168 bin: 4600
trial: 3 cell: 177 bin: 4600
trial: 3 cell: 180 bin: 4600
trial: 3 cell: 102 bin: 4800
trial: 3 cell: 171 bin: 4800
trial: 3 cell: 20 bin: 5000
trial: 3 cell: 66 bin: 5000
trial: 3 cell: 118 bin: 5000
trial: 3 cell: 171 bin: 5000
trial: 3 cell: 88 bin: 5200
trial: 3 cell: 121 bin: 5200
trial: 3 cell: 255 bin: 5200
trial: 3 cell: 88 bin: 5400
trial: 3 cell: 141 bin: 5400
trial: 3 cell: 159 bin: 5400
trial: 3 cell: 226 bin: 5400
trial: 3 cell: 232 bin: 5400
trial: 3 cell: 35 bin: 5600
trial: 3 cell: 42 bin: 5600
trial: 3 cell: 66 bin: 5600
trial: 3 cell: 87 bin: 5600
trial: 3 cell: 222 bin: 5600
trial: 3 cell: 247 bin: 5600
trial: 3 cell: 34 bin: 5800
trial: 3 cell: 35 bin: 5800
trial: 3 cell: 106 bin: 5800
trial: 3 cell: 222 bin: 5800
trial: 3 cell: 255 bin: 5800
trial: 3 cell: 56 bin: 6000
trial: 3 cell: 125 bin: 6000
trial: 3 cell: 154 bin: 6000
trial: 3 cell: 190 bin: 6000
trial: 3 cell: 192 bin: 6000
trial: 3 cell: 206 bin: 6

trial: 5 cell: 97 bin: 4000
trial: 5 cell: 178 bin: 4000
trial: 5 cell: 205 bin: 4000
trial: 5 cell: 243 bin: 4000
trial: 5 cell: 244 bin: 4000
trial: 5 cell: 58 bin: 4200
trial: 5 cell: 97 bin: 4200
trial: 5 cell: 167 bin: 4200
trial: 5 cell: 41 bin: 4400
trial: 5 cell: 105 bin: 4400
trial: 5 cell: 57 bin: 4600
trial: 5 cell: 134 bin: 4600
trial: 5 cell: 18 bin: 4800
trial: 5 cell: 90 bin: 4800
trial: 5 cell: 210 bin: 4800
trial: 5 cell: 28 bin: 5000
trial: 5 cell: 90 bin: 5000
trial: 5 cell: 92 bin: 5000
trial: 5 cell: 171 bin: 5200
trial: 5 cell: 177 bin: 5200
trial: 5 cell: 102 bin: 5400
trial: 5 cell: 168 bin: 5400
trial: 5 cell: 180 bin: 5400
trial: 5 cell: 234 bin: 5400
trial: 5 cell: 66 bin: 5600
trial: 5 cell: 96 bin: 5600
trial: 5 cell: 113 bin: 5600
trial: 5 cell: 168 bin: 5600
trial: 5 cell: 204 bin: 5600
trial: 5 cell: 43 bin: 5800
trial: 5 cell: 68 bin: 5800
trial: 5 cell: 113 bin: 5800
trial: 5 cell: 225 bin: 5800
trial: 5 cell: 54 bin: 6000
trial: 5 cell: 89 bin: 6000
t

trial: 7 cell: 131 bin: 600
trial: 7 cell: 137 bin: 600
trial: 7 cell: 142 bin: 600
trial: 7 cell: 26 bin: 800
trial: 7 cell: 73 bin: 800
trial: 7 cell: 76 bin: 800
trial: 7 cell: 95 bin: 800
trial: 7 cell: 132 bin: 800
trial: 7 cell: 222 bin: 800
trial: 7 cell: 214 bin: 1000
trial: 7 cell: 219 bin: 1000
trial: 7 cell: 224 bin: 1400
trial: 7 cell: 261 bin: 1400
trial: 7 cell: 74 bin: 1600
trial: 7 cell: 98 bin: 1600
trial: 7 cell: 152 bin: 1600
trial: 7 cell: 159 bin: 1600
trial: 7 cell: 189 bin: 1600
trial: 7 cell: 70 bin: 1800
trial: 7 cell: 86 bin: 1800
trial: 7 cell: 94 bin: 1800
trial: 7 cell: 161 bin: 1800
trial: 7 cell: 162 bin: 1800
trial: 7 cell: 189 bin: 1800
trial: 7 cell: 262 bin: 1800
trial: 7 cell: 93 bin: 2000
trial: 7 cell: 189 bin: 2000
trial: 7 cell: 73 bin: 2200
trial: 7 cell: 86 bin: 2200
trial: 7 cell: 134 bin: 2200
trial: 7 cell: 51 bin: 2400
trial: 7 cell: 58 bin: 2400
trial: 7 cell: 222 bin: 2600
trial: 7 cell: 159 bin: 2800
trial: 7 cell: 107 bin: 3000
trial: 7

...

trial: 20 cell: 211 bin: 8200
trial: 20 cell: 218 bin: 8200
trial: 20 cell: 89 bin: 8400
trial: 20 cell: 114 bin: 8400
trial: 20 cell: 114 bin: 8600
trial: 20 cell: 186 bin: 8600
trial: 20 cell: 218 bin: 8600
trial: 21 cell: 130 bin: 200
trial: 21 cell: 216 bin: 400
trial: 21 cell: 231 bin: 400
trial: 21 cell: 134 bin: 800
trial: 21 cell: 86 bin: 1000
trial: 21 cell: 109 bin: 1000
trial: 21 cell: 134 bin: 1000
trial: 21 cell: 86 bin: 1200
trial: 21 cell: 135 bin: 1200
trial: 21 cell: 189 bin: 1200
trial: 21 cell: 213 bin: 1200
trial: 21 cell: 64 bin: 1400
trial: 21 cell: 93 bin: 1400
trial: 21 cell: 208 bin: 1600
trial: 21 cell: 28 bin: 1800
trial: 21 cell: 102 bin: 1800
trial: 21 cell: 250 bin: 1800
trial: 21 cell: 41 bin: 2000
trial: 21 cell: 64 bin: 2000
trial: 21 cell: 224 bin: 2000
trial: 21 cell: 239 bin: 2000
trial: 21 cell: 246 bin: 2000
trial: 21 cell: 49 bin: 2200
trial: 21 cell: 64 bin: 2200
trial: 21 cell: 107 bin: 2200
trial: 21 cell: 128 bin: 2200
trial: 21 cell: 184 bin:

trial: 23 cell: 142 bin: 1200
trial: 23 cell: 214 bin: 1200
trial: 23 cell: 76 bin: 1400
trial: 23 cell: 132 bin: 1400
trial: 23 cell: 149 bin: 1400
trial: 23 cell: 158 bin: 1400
trial: 23 cell: 210 bin: 1400
trial: 23 cell: 214 bin: 1400
trial: 23 cell: 154 bin: 1800
trial: 23 cell: 214 bin: 1800
trial: 23 cell: 180 bin: 2000
trial: 23 cell: 205 bin: 2000
trial: 23 cell: 37 bin: 2200
trial: 23 cell: 161 bin: 2200
trial: 23 cell: 189 bin: 2200
trial: 23 cell: 205 bin: 2200
trial: 23 cell: 88 bin: 2400
trial: 23 cell: 161 bin: 2400
trial: 23 cell: 189 bin: 2400
trial: 23 cell: 73 bin: 2600
trial: 23 cell: 86 bin: 2600
trial: 23 cell: 98 bin: 2600
trial: 23 cell: 238 bin: 2600
trial: 23 cell: 213 bin: 3000
trial: 23 cell: 152 bin: 3200
trial: 23 cell: 86 bin: 3400
trial: 23 cell: 134 bin: 3400
trial: 23 cell: 145 bin: 3400
trial: 23 cell: 38 bin: 3600
trial: 23 cell: 58 bin: 3600
trial: 23 cell: 134 bin: 3600
trial: 23 cell: 185 bin: 3600
trial: 23 cell: 208 bin: 3600
trial: 23 cell: 58 

trial: 25 cell: 76 bin: 800
trial: 25 cell: 132 bin: 800
trial: 25 cell: 98 bin: 1000
trial: 25 cell: 145 bin: 1000
trial: 25 cell: 152 bin: 1000
trial: 25 cell: 214 bin: 1000
trial: 25 cell: 86 bin: 1400
trial: 25 cell: 98 bin: 1400
trial: 25 cell: 233 bin: 1400
trial: 25 cell: 94 bin: 1600
trial: 25 cell: 135 bin: 1600
trial: 25 cell: 175 bin: 1600
trial: 25 cell: 185 bin: 1600
trial: 25 cell: 98 bin: 1800
trial: 25 cell: 185 bin: 1800
trial: 25 cell: 238 bin: 1800
trial: 25 cell: 250 bin: 1800
trial: 25 cell: 28 bin: 2200
trial: 25 cell: 64 bin: 2200
trial: 25 cell: 89 bin: 2200
trial: 25 cell: 107 bin: 2200
trial: 25 cell: 128 bin: 2200
trial: 25 cell: 150 bin: 2200
trial: 25 cell: 213 bin: 2200
trial: 25 cell: 224 bin: 2200
trial: 25 cell: 246 bin: 2200
trial: 25 cell: 21 bin: 2400
trial: 25 cell: 27 bin: 2400
trial: 25 cell: 45 bin: 2400
trial: 25 cell: 64 bin: 2400
trial: 25 cell: 87 bin: 2400
trial: 25 cell: 102 bin: 2400
trial: 25 cell: 45 bin: 2600
trial: 25 cell: 64 bin: 260

trial: 27 cell: 120 bin: 400
trial: 27 cell: 76 bin: 800
trial: 27 cell: 109 bin: 800
trial: 27 cell: 132 bin: 800
trial: 27 cell: 143 bin: 800
trial: 27 cell: 152 bin: 800
trial: 27 cell: 214 bin: 800
trial: 27 cell: 229 bin: 800
trial: 27 cell: 98 bin: 1000
trial: 27 cell: 98 bin: 1200
trial: 27 cell: 145 bin: 1200
trial: 27 cell: 86 bin: 1400
trial: 27 cell: 52 bin: 1600
trial: 27 cell: 175 bin: 1600
trial: 27 cell: 185 bin: 1600
trial: 27 cell: 213 bin: 1800
trial: 27 cell: 246 bin: 1800
trial: 27 cell: 45 bin: 2000
trial: 27 cell: 50 bin: 2000
trial: 27 cell: 122 bin: 2000
trial: 27 cell: 45 bin: 2200
trial: 27 cell: 49 bin: 2200
trial: 27 cell: 89 bin: 2200
trial: 27 cell: 136 bin: 2400
trial: 27 cell: 150 bin: 2400
trial: 27 cell: 208 bin: 2400
trial: 27 cell: 42 bin: 2600
trial: 27 cell: 69 bin: 2600
trial: 27 cell: 87 bin: 2600
trial: 27 cell: 89 bin: 2600
trial: 27 cell: 91 bin: 2600
trial: 27 cell: 41 bin: 2800
trial: 27 cell: 69 bin: 2800
trial: 27 cell: 112 bin: 2800
trial

trial: 29 cell: 89 bin: 2200
trial: 29 cell: 150 bin: 2200
trial: 29 cell: 38 bin: 2400
trial: 29 cell: 41 bin: 2400
trial: 29 cell: 69 bin: 2400
trial: 29 cell: 128 bin: 2400
trial: 29 cell: 182 bin: 2400
trial: 29 cell: 38 bin: 2600
trial: 29 cell: 42 bin: 2600
trial: 29 cell: 69 bin: 2600
trial: 29 cell: 107 bin: 2600
trial: 29 cell: 128 bin: 2600
trial: 29 cell: 42 bin: 2800
trial: 29 cell: 91 bin: 2800
trial: 29 cell: 150 bin: 2800
trial: 29 cell: 230 bin: 2800
trial: 29 cell: 243 bin: 2800
trial: 29 cell: 92 bin: 3000
trial: 29 cell: 124 bin: 3000
trial: 29 cell: 153 bin: 3000
trial: 29 cell: 210 bin: 3000
trial: 29 cell: 83 bin: 3200
trial: 29 cell: 90 bin: 3200
trial: 29 cell: 97 bin: 3200
trial: 29 cell: 68 bin: 3400
trial: 29 cell: 113 bin: 3600
trial: 29 cell: 136 bin: 3600
trial: 29 cell: 247 bin: 3600
trial: 29 cell: 19 bin: 3800
trial: 29 cell: 59 bin: 3800
trial: 29 cell: 66 bin: 3800
trial: 29 cell: 196 bin: 3800
trial: 29 cell: 213 bin: 3800
trial: 29 cell: 219 bin: 38